In [7]:
#from langchain.chat_models import ChatOllama
#chat = ChatOllama(model="mistral:latest")
#a = chat.predict("How many planets are there?")
#a

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-1106")
chat = ChatOpenAI()
b = chat.predict("How many planets are there?")
a = llm.predict("How many planets are there?")

a,b

d:\FULLSTACK-GPT\env\lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
d:\FULLSTACK-GPT\env\lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors..


('Traditionally, there are 8 planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. However, some astronomers consider Pluto to be the 9th planet, while others do not.',
 'There are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')

In [9]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You are a geography expert. And you only reply in Korean"),
    AIMessage(content="안녕하세요, 제 이름은 기며노입니다."),
    HumanMessage(content="What is distance between Mexico and Thailand. Also, what is your name?")
]

chat.predict_messages(messages)

AIMessage(content='멕시코와 태국 사이의 거리는 대략 15,000km입니다. 제 이름은 기며노입니다.')

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template(
    "{country_a}와 {country_b}의 거리를 알려줘."
)

prompt = template.format(country_a="대한민국", country_b="일본")

chat = ChatOpenAI(temperature=0.1)

chat.predict(prompt)


'대한민국과 일본 사이의 거리는 직선거리로 약 1,000km 정도입니다. 두 나라 사이에는 동해를 사이에 두고 있어서 해상거리로는 더 멀게 느껴질 수 있습니다. 비행기로 이동할 경우 약 2시간 정도 소요되며, 선박으로 이동할 경우에는 약 1일 정도 소요될 수 있습니다.'

In [14]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "안녕, 내 이름은 {name}!"),
    ("human", "너의 이름은 뭐야? 그리고 {country_a}와 {country_b}의 거리를 알려줘."),
])

prompt = template.format_messages(
    language="Korean",
    name="소크라테스",
    country_a="마카오",
    country_b="서울"
)

chat.predict_messages(prompt)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors..


AIMessage(content='내 이름은 소크라테스야. 마카오와 서울 사이의 거리는 대략 2,500km 정도 되어요.')

In [17]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommaOutputParser()

p.parse("Hello,how, are, you")



['Hello', 'how', ' are', ' you']

In [20]:
template = ChatPromptTemplate.from_messages([
    ("system", "넌 list를 만드는 기ㅖ야. 네 대답은 최대 {max_items}개의 원소를 가진 list로 반환될거야. 다른 것은 대답하지마."),
    ("human", "{question}"),
])


prompt = template.format_messages(
    max_items = 10,
    question="색깔이 뭐야?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

['빨간색', ' 파란색', ' 노란색', ' 초록색', ' 보라색', ' 주황색', ' 검은색', ' 흰색', ' 회색', ' 핑크색']

In [22]:
chain = template | chat | CommaOutputParser()

chain.invoke({"max_items":5, "question":"포켓몬은 뭐야?"})

["['피카츄'", " '파이리'", " '꼬부기'", " '이상해씨'", " '리자몽']"]

In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature= 0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "너는 세계적인 쉐프야. 너는 따라하기 쉬운 레시피를 만들어줘야해. 그리고 그 레시피는 찾기 쉬운 재료로 구성되어야해."),
         ("human",
          "나는 {cuisine}요리를 요리하고 싶어."),
    ]
)

chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system","너는 채식요리사야. 너는 대체할 재료와 준비방법을 찾아야하고 기존 요리법을 크게 바꿔서는 안돼. 만약 대체할 것이 없다면 모르겠다고 대답해."),
        ("human", "{recipe}")
    ]
)

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine":"한식"})

좋아요! 한식을 요리하는 것은 정말 즐거운 경험이 될 거예요. 이 간단하면서 맛있는 레시피를 추천해드릴게요. 바로 떡볶이입니다. 떡볶이는 한국에서 매우 인기 있는 길거리 음식 중 하나로, 매콤하고 달콤한 맛이 일품이에요. 준비물은 다음과 같아요:

- 떡 2컵
- 양파 1개
- 대파 1대
- 어묵 100g
- 고추장 3큰술
- 고춧가루 1큰술
- 설탕 2큰술
- 국물용 멸치 5마리
- 물 4컵

이제 레시피를 시작해볼까요?

1. 먼저 냄비에 물 4컵과 멸치를 넣고 끓여주세요. 멸치 육수를 내기 위한 과정이에요.
2. 양파와 대파는 얇게 채 썰어주세요. 어묵은 한입 크기로 잘라주세요.
3. 끓는 물에 고추장, 고춧가루, 설탕을 넣고 잘 섞어주세요.
4. 양파, 대파, 어묵, 떡을 넣고 끓여주세요.
5. 멸치 육수를 넣고 중불로 끓여주세요.
6. 떡이 부드러워지면 불을 끄고 완성되었어요.

이제 맛있는 떡볶이가 완성되었어요. 매운 맛을 원하시면 고춧가루를 더 넣어주세요. 즐거운 한식 요리 시간 되세요!죄송해요, 하지만 떡볶이 레시피를 대체할 재료나 방법을 찾기 어려워요. 이 레시피는 전통적인 한식 레시피이기 때문에 크게 바꾸기 어려울 것 같아요. 만약 다른 요리에 대해 궁금한 점이 있거나 다른 도움이 필요하시다면 언제든지 물어주세요!

AIMessageChunk(content='죄송해요, 하지만 떡볶이 레시피를 대체할 재료나 방법을 찾기 어려워요. 이 레시피는 전통적인 한식 레시피이기 때문에 크게 바꾸기 어려울 것 같아요. 만약 다른 요리에 대해 궁금한 점이 있거나 다른 도움이 필요하시다면 언제든지 물어주세요!')

In [ ]:
#11 MEETINGGPT

import subprocess

def extract_audio_from_video(video_path, audio_path):
    command = ["ffmpeg" "-i" "files/podcast.mp4" "-vn" "files/audio.mp3"]

#ffmpeg 다운로드.
    
#pydub